In [0]:
# boilerplate

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.window import *

# create spark session
# /Workspace/Users/datakraft867@gmail.com/books.csv
spark = SparkSession.builder.appName("books_query").getOrCreate()
books_df = spark.read.csv("/Volumes/datakraft_batch1/default/datasets/books.csv", header=True, inferSchema=True)

In [0]:
books_df.createOrReplaceTempView("books_tbl")

In [0]:
# count the number of rows
books_df.count()

11127

In [0]:
# display top 10 rows
display(books_df.limit(10))


num_pages,authors,average_rating,bookID,isbn,isbn13,language_code,publication_date,publisher,ratings_count,text_reviews_count,title
652,J.K. Rowling/Mary GrandPré,4.57,1,0439785960,9780439785969,eng,9/16/2006,Scholastic Inc.,2095690,27591,Harry Potter and the Half-Blood Prince (Harry Potter #6)
870,J.K. Rowling/Mary GrandPré,4.49,2,0439358078,9780439358071,eng,9/1/2004,Scholastic Inc.,2153167,29221,Harry Potter and the Order of the Phoenix (Harry Potter #5)
352,J.K. Rowling,4.42,4,0439554896,9780439554893,eng,11/1/2003,Scholastic,6333,244,Harry Potter and the Chamber of Secrets (Harry Potter #2)
435,J.K. Rowling/Mary GrandPré,4.56,5,043965548X,9780439655484,eng,5/1/2004,Scholastic Inc.,2339585,36325,Harry Potter and the Prisoner of Azkaban (Harry Potter #3)
2690,J.K. Rowling/Mary GrandPré,4.78,8,0439682584,9780439682589,eng,9/13/2004,Scholastic,41428,164,Harry Potter Boxed Set Books 1-5 (Harry Potter #1-5)
152,W. Frederick Zimmerman,3.74,9,0976540606,9780976540601,en-US,4/26/2005,Nimble Books,19,1,Unauthorized Harry Potter Book Seven News: Half-Blood Prince Analysis and Speculation
3342,J.K. Rowling,4.73,10,0439827604,9780439827607,eng,9/12/2005,Scholastic,28242,808,Harry Potter Collection (Harry Potter #1-6)
815,Douglas Adams,4.38,12,0517226952,9780517226957,eng,11/1/2005,Gramercy Books,3628,254,The Ultimate Hitchhiker's Guide: Five Complete Novels and One Story (Hitchhiker's Guide to the Galaxy #1-5)
815,Douglas Adams,4.38,13,0345453743,9780345453747,eng,4/30/2002,Del Rey Books,249558,4080,The Ultimate Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy #1-5)
215,Douglas Adams,4.22,14,1400052920,9781400052929,eng,8/3/2004,Crown,4930,460,The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy #1)


In [0]:
books_df.show(10)

+---------+--------------------+--------------+------+----------+-------------+-------------+----------------+---------------+-------------+------------------+--------------------+
|num_pages|             authors|average_rating|bookID|      isbn|       isbn13|language_code|publication_date|      publisher|ratings_count|text_reviews_count|               title|
+---------+--------------------+--------------+------+----------+-------------+-------------+----------------+---------------+-------------+------------------+--------------------+
|      652|J.K. Rowling/Mary...|          4.57|     1|0439785960|9780439785969|          eng|       9/16/2006|Scholastic Inc.|      2095690|             27591|Harry Potter and ...|
|      870|J.K. Rowling/Mary...|          4.49|     2|0439358078|9780439358071|          eng|        9/1/2004|Scholastic Inc.|      2153167|             29221|Harry Potter and ...|
|      352|        J.K. Rowling|          4.42|     4|0439554896|9780439554893|          eng|  

In [0]:
# How many total books are present in the database?
books_cnt = spark.sql(\
    f"""
    SELECT COUNT(isbn) as books_cnt FROM books_tbl
    """).show()

+---------+
|books_cnt|
+---------+
|    11127|
+---------+



In [0]:
# what is the total page count of all books published by Bill Bryson?
# cast num_pages to integer
books_df = books_df.withColumn("num_pages", col("num_pages").cast("integer"))
# verify
books_df.printSchema()
# sql
tot_pg_bb = spark.sql(\
    f"""
    SELECT SUM(num_pages)
    FROM books_tbl
    WHERE authors = "Bill Bryson"
    """).show()

root
 |-- num_pages: integer (nullable = true)
 |-- authors: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- bookID: integer (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- publication_date: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- ratings_count: integer (nullable = true)
 |-- text_reviews_count: integer (nullable = true)
 |-- title: string (nullable = true)

+--------------+
|sum(num_pages)|
+--------------+
|        6865.0|
+--------------+



In [0]:
# for books published in 1943, what is the average book rating?
# extract year from year from publication date
avg_rating = spark.sql(\
  f"""
  WITH CTE AS (
  SELECT 
  CAST(RIGHT(publication_date, 4) AS INT) AS yr,
  ROUND(AVG(average_rating),2) AS avg_rating
  FROM books_tbl
  GROUP BY 1
  ORDER BY 1
  )

  SELECT yr, avg_rating FROM CTE WHERE yr = 1943;
  """).show()

+----+----------+
|  yr|avg_rating|
+----+----------+
|1943|      4.21|
+----+----------+



In [0]:
# Oxford University Press and published in 2002, list the books? 

publisher = spark.sql(\
f"""
WITH CTE AS (
SELECT publisher, title,
CAST(RIGHT(publication_date, 4) AS INT) AS yr
FROM books_tbl
)
SELECT * FROM CTE WHERE yr = 1998 and publisher like '%Oxford University Press%';
""").display()

publisher,title,yr
Oxford University Press,War and Peace,1998
Oxford University Press,The Histories,1998
Oxford University Press,Antigone; Oedipus the Kingn; Electra,1998
Oxford University Press USA,Moby Dick: or The White Whale (Oxford Illustrated Classics),1998
Oxford University Press,The Bostonians,1998
Oxford University Press,The Canterbury Tales,1998
Oxford University Press,Waverley,1998
Oxford University Press,The Karamazov Brothers,1998
Oxford University Press,The Castle of Otranto,1998
Oxford University Press USA,Faust Part One,1998


In [0]:
#Which publisher had the books with the highest rating; # publisher, book_title, rating
rating = spark.sql(\
    f"""
    SELECT publisher,title as book_title,avg(average_rating)as rating
    from books_tbl
    GROUP BY publisher,book_title
    ORDER BY rating desc
    """).display()
     


publisher,book_title,rating
Vantage Press,Bulgakov's the Master and Margarita: The Text as a Cipher,5.0
BradyGames,Zone of the Enders: The 2nd Runner Official Strategy Guide,5.0
University Press of New England,Oliver Wendell Holmes in Paris: Medicine Theology and the Autocrat of the Breakfast Table,5.0
Square One Publishers,The Diamond Color Meditation: Color Pathway to the Soul,5.0
Disney Press,Tyrannosaurus Wrecks (Stanley #1),5.0
Multnomah,His Princess Devotional: A Royal Encounter With Your King,5.0
Oxford University Press USA,Comoediae 1: Acharenses/Equites/Nubes/Vespae/Pax/Aves,5.0
BBC Physical Audio,The Goon Show Volume 4: My Knees Have Fallen Off!,5.0
Chartwell Books,Winchester Shotguns,5.0
Courage Books,The New Big Book of America,5.0


In [0]:
# How many books did Dan Brown publish? # all books published by Dan Brown (even as co-author) # author, book_title, publication_date

Dan_Brown = spark.sql(
    """
SELECT authors,title as book_title , publication_date
FROM books_tbl
WHERE authors LIKE 'Dan Brown' or publisher = 'Dan Brown'
 """
).display()


authors,book_title,publication_date
Dan Brown,Angels & Demons (Robert Langdon #1),4/1/2006
Dan Brown,Ángeles y demonios (Robert Langdon #1),12/1/2005
Dan Brown,The Da Vinci Code (Robert Langdon #2),3/28/2006
Dan Brown,The Da Vinci Code,3/28/2006
Dan Brown,Deception Point,4/1/2006
Dan Brown,Deception Point,5/1/2004
Dan Brown,Deception Point,8/1/2005
Dan Brown,The Da Vinci Code (Robert Langdon #2),3/28/2006
Dan Brown,Angels & Demons (Robert Langdon #1),5/3/2005
Dan Brown,The Da Vinci Code,3/28/2006


In [0]:
# which book had highest text_reviews_count? # book_tile, text_reviews_count
text_reviews = spark.sql("""
    SELECT title as book_title, text_reviews_count
    FROM books_tbl
    WHERE text_reviews_count = (
        SELECT MAX(text_reviews_count)
        FROM books_tbl
    )
""").display()

book_title,text_reviews_count
Twilight (Twilight #1),94265
